Este fichero contiene el método **buscar_una_noticia** que se encarga de **buscar todas las noticias con las que está relacionada la noticia buscada**. Por argumento se recibe el periódico y el número de la noticia buscada. Los pasos que sigue esta función están explicados paso a paso en el fichero **buscar_una_noticia.ipynb**.

Además, al final de este fichero se llama al método **buscar_una_noticia** para **todas las noticias de la base de datos**, realizando así el primer filtrado. Tras esto, lo que se obtiene es **cada noticia con que otras noticias está relacionadas**, de esta manera será **más fácil encontrar conjuntos más pequeños de noticias relacionadas**.

In [1]:
# Metodo que se encarga de buscar las noticias con las que tiene relacion la noticia buscada
def buscar_una_noticia(periodico_buscado, num_noticia_buscada):
    # Librerias utilizadas
    import json
    import os
    
    # Guardo en una variable todas las noticias del periodico_buscado que he obtenido
    noticias_periodico_buscado = {}
    url_noticias_periodico_buscado = 'Noticias/' + periodico_buscado + '/noticias.json'
    with open(url_noticias_periodico_buscado) as file:
        noticias_periodico_buscado = json.load(file)
    
    # Guardo las noticias recopiladas de cada periodico en una variable diferente
    # EL MUNDO
    noticias_periodico_ELMUNDO = {}
    url_noticias_periodico_ELMUNDO = 'Noticias/EL MUNDO/noticias.json'
    with open(url_noticias_periodico_ELMUNDO) as file:
        noticias_periodico_ELMUNDO = json.load(file)

    # 20 minutos
    noticias_periodico_20minutos = {}
    url_noticias_periodico_20minutos = 'Noticias/20 minutos/noticias.json'
    with open(url_noticias_periodico_20minutos) as file:
        noticias_periodico_20minutos = json.load(file)

    # elEconomista.es
    noticias_periodico_elEconomistaes = {}
    url_noticias_periodico_elEconomistaes = 'Noticias/elEconomista.es/noticias.json'
    with open(url_noticias_periodico_elEconomistaes) as file:
        noticias_periodico_elEconomistaes = json.load(file)

    # La Informacion
    noticias_periodico_LaInformacion = {}
    url_noticias_periodico_LaInformacion = 'Noticias/La Informacion/noticias.json'
    with open(url_noticias_periodico_LaInformacion) as file:
        noticias_periodico_LaInformacion = json.load(file)

    # El Confidencial
    noticias_periodico_ElConfidencial = {}
    url_noticias_periodico_ElConfidencial = 'Noticias/El Confidencial/noticias.json'
    with open(url_noticias_periodico_ElConfidencial) as file:
        noticias_periodico_ElConfidencial = json.load(file)
    
    # Librerias utilizadas para construir la matriz de distancias
    import numpy as np
    from ipynb.fs.full.NCD import NCD # Funcion para calcular la NCD
    from ipynb.fs.full.NCD import write_file_str #Funcion para escribir el contenido de una noticia a un fichero txt

    # Escribo el contenido de la noticia buscada en un fichero txt para compararla con las otras noticias
    filename_noticia_buscada = "noticia_buscada.txt" # Fichero txt para escribir el contenido de la noticia buscada
    longitud_busqueda = len(noticias_periodico_buscado[num_noticia_buscada]['contenido']) # Tamaño noticia buscada
    nombre_noticia_buscada = periodico_buscado + "-" + num_noticia_buscada

    # Compruebo que la noticia buscada no es demasiado corta
    indicador_noticia_demasiado_corta = 0
    if longitud_busqueda < 1250:
        indicador_noticia_demasiado_corta += 1
        return # No hay que calcular nada, se descarta por ser demasiado corta
    
    # Escribo el contenido de la noticia en un fichero
    write_file_str(filename_noticia_buscada, noticias_periodico_buscado[num_noticia_buscada]['contenido']) 

    # Fichero txt para escribir el contenido de la noticia que voy a comparar
    filename_noticia_comparada = "noticia_comparada.txt" 

    dicc_periodico_noticias = {} # Diccionario para almacenar las noticias de cada periodico
    dicc_periodico_noticias['EL MUNDO'] = noticias_periodico_ELMUNDO
    dicc_periodico_noticias['20 minutos'] = noticias_periodico_20minutos
    dicc_periodico_noticias['elEconomista.es'] = noticias_periodico_elEconomistaes
    dicc_periodico_noticias['La Informacion'] = noticias_periodico_LaInformacion
    dicc_periodico_noticias['El Confidencial'] = noticias_periodico_ElConfidencial


    # Variables para hacer un estudio de las noticias que finalmente se han comparado
    dicc_periodico_num_noticias = {} # Se va a almacenar para cada periodico el numero de noticias comparadas con la buscada
    dicc_periodico_num_noticias['EL MUNDO'] = 0
    dicc_periodico_num_noticias['20 minutos'] = 0
    dicc_periodico_num_noticias['elEconomista.es'] = 0
    dicc_periodico_num_noticias['La Informacion'] = 0
    dicc_periodico_num_noticias['El Confidencial'] = 0

    matriz_NCDs = {} # Diccionario donde voy a ir guardando los valores de las distancias
    contador_dependencias = 0 # Cuenta el numero de noticias que son parecidas a la noticia buscada

    # Recorro para cada periodico todas las noticias
    for periodico in dicc_periodico_noticias:
        for noticia in dicc_periodico_noticias[periodico]:
            # Comparo la noticia con la noticia buscada (solo si no es la misma)
            if periodico != periodico_buscado or noticia != num_noticia_buscada:

                longitud_comparada = len(dicc_periodico_noticias[periodico][noticia]['contenido']) # Tamaño noticia comparada

                # Solo comparo la noticia si su contenido no esta vacio y si tiene una longitud mayor de 1250
                if dicc_periodico_noticias[periodico][noticia]['contenido'] != "" and longitud_comparada >= 1250:
                    # Ademas tiene que tener una longitud parecida con la noticia buscada
                    if longitud_busqueda + 1000 >= longitud_comparada and longitud_busqueda - 1000 <= longitud_comparada:

                        dicc_periodico_num_noticias[periodico] += 1 # Cuento las noticias comparadas
                        nombre_noticia_comparada = periodico + "-" + noticia

                        # Escribo el contenido de la noticia comparada en un fichero txt para compararla con la noticia buscada
                        write_file_str(filename_noticia_comparada, dicc_periodico_noticias[periodico][noticia]['contenido']) # Escribo el contenido en un fichero

                        # Calculo las distancias de compresion normalizadas: NCD(noticia_buscada, noticia_comparada) y NCD(noticia_comparada, noticia_buscada)
                        NCDs = NCD(filename_noticia_buscada, filename_noticia_comparada)

                        # Quiero de alguna manera identificar las noticias con mas similitud a otras y las que menos
                        # Para ello voy a contar las veces que:
                        # n1 --> n2 < 0,765 y n2 --> n1 < 0,8
                        # o
                        # n2 --> n1 < 0,765 y n1 --> n2 < 0,8
                        # o
                        # n2 --> n1 < 0,765 y n1 --> n2 < 0,765
                        # Es decir, al menos una de las distancias debe ser < 0,765 y la otra < 0,8
                        if ((NCDs[0] < 0.765 and NCDs[1] < 0.8) or (NCDs[1] < 0.765 and NCDs[0] < 0.8)):
                            contador_dependencias += 1 # Aumento el valor de noticias similares encontradas

                        # Guardo los resultados en la matriz de distancias

                        # noticia_buscada - noticia_comparada
                        if nombre_noticia_buscada not in matriz_NCDs:
                            matriz_NCDs[nombre_noticia_buscada] = {}
                            matriz_NCDs[nombre_noticia_buscada][nombre_noticia_comparada] = NCDs[0]
                        else:
                            matriz_NCDs[nombre_noticia_buscada][nombre_noticia_comparada] = NCDs[0]

                        # noticia_comparada - noticia_buscada
                        if nombre_noticia_comparada not in matriz_NCDs:
                            matriz_NCDs[nombre_noticia_comparada] = {}
                            matriz_NCDs[nombre_noticia_comparada][nombre_noticia_buscada] = NCDs[1]
                        else:
                            matriz_NCDs[nombre_noticia_comparada][nombre_noticia_buscada] = NCDs[1]
                            
    import math

    # Calculo el numero de noticias que se han comparado
    # Antes compruebo que se haya comparado con alguna noticia
    if nombre_noticia_buscada not in matriz_NCDs:
        return #No se ha podido comparar por el tamaño, acaba la funcion
    numero_noticias_comparadas = len(matriz_NCDs[nombre_noticia_buscada])
    # Para que el resultado sea legible es necesario dividir las distancias en varias matrices de distancia
    # Se ha decidido que para un conjunto de 50 noticias el dendograma resultando se puede estudiar bien
    # Hay que tener en cuenta que en todas las matrices debe estar la noticia buscada, ya que es la que se esta comparando
    numero_matrices_distancia = numero_noticias_comparadas / 50

    # Creo las matrices de distancia que sean necesarias

    list_matrices = [] # lista de matrices
    list_posiciones = [] # lista de posiciones en cada matriz
    contador_noticias_comparadas_matriz = 0 # Indica el numero de noticias comparadas que ya han sido añadidas en matrices

    for numero_matriz in range(0,math.ceil(numero_matrices_distancia)):

        matriz_distancias_noticia_buscada_sin_numpy = [] # Estrutura para crear la matriz de distancias

        # Contenido inicial de cada fila de la matriz, tiene 50 (noticia_buscada + 49 noticias_comparadas) columnas
        # Primero pongo todo a 1
        for fila in range(0, 50):
            list_aux = []
            for columna in range(0, 50):  
                list_aux.append(1)
            matriz_distancias_noticia_buscada_sin_numpy.append(list_aux)
            
        # Es necesario guardar las posiciones porque en el dendograma final aparecen numeradas las noticias segun
        # su posicion en la matriz, pero no se puede identificar la noticia que es

        # Guarda la posicion de cada noticia en la matriz
        posicion_noticia_matriz = [nombre_noticia_buscada] # la primera siempre es la noticia_buscada 

        # Guardo la posicion de cada noticia en la matriz
        aux_contador_noticias_comparadas_matriz = contador_noticias_comparadas_matriz
        limite = contador_noticias_comparadas_matriz + 49 # Indica hasta que noticia_comparada tengo que leer
        cont = 1
        for nombre_noticia_comparada in matriz_NCDs[nombre_noticia_buscada]:
            if cont > limite:
                break
            if cont > contador_noticias_comparadas_matriz:
                posicion_noticia_matriz.append(nombre_noticia_comparada)
                contador_noticias_comparadas_matriz += 1
            cont += 1

        # Relleno con ceros la diagonal de la matriz ya que una noticia y la misma, son iguales
        cont_fila = 0
        for fila in matriz_distancias_noticia_buscada_sin_numpy:
            fila[cont_fila] = 0
            cont_fila += 1

        # Relleno con las distancias calculadas

        # Primero noticia_buscada --> noticia_comparada
        cont_columna = 1
        cont = 1
        for nombre_noticia_comparada in matriz_NCDs[nombre_noticia_buscada]:
            if cont > limite:
                break
            if cont > aux_contador_noticias_comparadas_matriz:
                matriz_distancias_noticia_buscada_sin_numpy[0][cont_columna] = matriz_NCDs[nombre_noticia_buscada][nombre_noticia_comparada]
                cont_columna += 1
            cont += 1

        # Segundo noticia_comparada --> noticia_buscada
        cont_fila = 1
        cont = 1
        for nombre_noticia_comparada in matriz_NCDs[nombre_noticia_buscada]:
            if cont > limite:
                break
            if cont > aux_contador_noticias_comparadas_matriz:
                matriz_distancias_noticia_buscada_sin_numpy[cont_fila][0] = matriz_NCDs[nombre_noticia_comparada][nombre_noticia_buscada]
                cont_fila += 1
            cont += 1

        
        matriz_distancias_noticia_buscada = np.array(matriz_distancias_noticia_buscada_sin_numpy)

        # Guardo la matriz y las posiciones
        list_matrices.append(matriz_distancias_noticia_buscada)
        list_posiciones.append(posicion_noticia_matriz)
        
    # Gráficos
    # ==============================================================================
    import matplotlib.pyplot as plt
    from matplotlib import style
    style.use('ggplot') or plt.style.use('ggplot')

    # Preprocesado y modelado
    # ==============================================================================
    from sklearn.cluster import AgglomerativeClustering
    from scipy.cluster.hierarchy import dendrogram
    from sklearn.preprocessing import scale
    from sklearn.metrics import silhouette_score

    # Ficheros donde guardar los datos si la noticia no es demasiado corta
    if indicador_noticia_demasiado_corta == 0:
        # Carpeta raiz de todos los datos
        if not os.path.exists('Noticias/Base de datos'):
            os.mkdir('Noticias/Base de datos')

        # Carpeta segun el numero de dependencias para diferencias segun las noticias con mas dependencias
        if not os.path.exists('Noticias/Base de datos/' + str(contador_dependencias)):
            os.mkdir('Noticias/Base de datos/' + str(contador_dependencias))

        # Carpeta para cada noticia buscada
        if not os.path.exists('Noticias/Base de datos/' + str(contador_dependencias) + '/' + nombre_noticia_buscada):
            os.mkdir('Noticias/Base de datos/' + str(contador_dependencias) + '/' + nombre_noticia_buscada)
        carpeta_noticia_buscada = 'Noticias/Base de datos/' + str(contador_dependencias) + '/' + nombre_noticia_buscada

    # Funcion para dibujar graficas
    def plot_dendrogram(model, **kwargs):
        '''
        Esta función extrae la información de un modelo AgglomerativeClustering
        y representa su dendograma con la función dendogram de scipy.cluster.hierarchy
        '''

        counts = np.zeros(model.children_.shape[0])
        n_samples = len(model.labels_)
        for i, merge in enumerate(model.children_):
            current_count = 0
            for child_idx in merge:
                if child_idx < n_samples:
                    current_count += 1  # leaf node
                else:
                    current_count += counts[child_idx - n_samples]
            counts[i] = current_count

        linkage_matrix = np.column_stack([model.children_, model.distances_,
                                          counts]).astype(float)

        # Plot
        dendrogram(linkage_matrix, **kwargs)

    # Creo un cluster por cada matriz de distancias
    for numero_matriz in range(0,math.ceil(numero_matrices_distancia)):
        # Modelo utilizado
        # ==============================================================================
        modelo_hclust_complete = AgglomerativeClustering(
                                    affinity = 'precomputed',
                                    linkage  = 'single',
                                    distance_threshold = 0,
                                    n_clusters         = None
                                )
        modelo_hclust_complete.fit(X=list_matrices[numero_matriz])

        # Dendrograma
        # ==============================================================================
        fig, figura = plt.subplots(1, 1, figsize=(20,8))
        plot_dendrogram(modelo_hclust_complete, color_threshold=0, ax=figura)
        figura.set_title("Distancia NCD, Linkage single, matriz " + str(numero_matriz))
        plt.tight_layout();

        # Guardo los datos
        # ==============================================================================
        # Solo guardo los datos si la noticia no es demasiado corta
        if indicador_noticia_demasiado_corta == 0:
            # Cluster
            nombre_imagen = carpeta_noticia_buscada + "/Cluster " + str(numero_matriz) + ".jpg"
            plt.savefig(nombre_imagen)
            # Matrices de distancia
            # Solo tiene distancias la primera fila y primera columna. Se compara la noticia_buscada con el resto
            nombre_matrices = carpeta_noticia_buscada + "/Matriz " + str(numero_matriz) + ".json"
            with open(nombre_matrices, 'w') as fp:
                json.dump(list_matrices[numero_matriz].tolist(), fp)

            # Lista de posiciones
            # Posicion de la lista = fila en la matriz, la primera fila es de la noticia_buscada
            nombre_posiciones = carpeta_noticia_buscada + "/Posicion " + str(numero_matriz) + ".json"
            with open(nombre_posiciones, 'w') as fp:
                json.dump(list_posiciones[numero_matriz], fp)


Se llama al método **buscar_una_noticia** para **todas las noticias de la base de datos**, realizando así el primer filtrado:

In [ ]:
# Genero la base de datos comparando todas las noticias
# Guardo las noticias recopiladas de cada periodico en una variable diferente
import json
import os

# EL MUNDO
noticias_periodico_ELMUNDO = {}
url_noticias_periodico_ELMUNDO = 'Noticias/EL MUNDO/noticias.json'
with open(url_noticias_periodico_ELMUNDO) as file:
    noticias_periodico_ELMUNDO = json.load(file)
    
# 20 minutos
noticias_periodico_20minutos = {}
url_noticias_periodico_20minutos = 'Noticias/20 minutos/noticias.json'
with open(url_noticias_periodico_20minutos) as file:
    noticias_periodico_20minutos = json.load(file)
    
# elEconomista.es
noticias_periodico_elEconomistaes = {}
url_noticias_periodico_elEconomistaes = 'Noticias/elEconomista.es/noticias.json'
with open(url_noticias_periodico_elEconomistaes) as file:
    noticias_periodico_elEconomistaes = json.load(file)
    
# La Informacion
noticias_periodico_LaInformacion = {}
url_noticias_periodico_LaInformacion = 'Noticias/La Informacion/noticias.json'
with open(url_noticias_periodico_LaInformacion) as file:
    noticias_periodico_LaInformacion = json.load(file)
    
# El Confidencial
noticias_periodico_ElConfidencial = {}
url_noticias_periodico_ElConfidencial = 'Noticias/El Confidencial/noticias.json'
with open(url_noticias_periodico_ElConfidencial) as file:
    noticias_periodico_ElConfidencial = json.load(file)
    

dicc_periodico_noticias = {} # Diccionario para almacenar las noticias de cada periodico
dicc_periodico_noticias['EL MUNDO'] = noticias_periodico_ELMUNDO # todas
dicc_periodico_noticias['20 minutos'] = noticias_periodico_20minutos # todas
dicc_periodico_noticias['elEconomista.es'] = noticias_periodico_elEconomistaes #todas
dicc_periodico_noticias['La Informacion'] = noticias_periodico_LaInformacion #todas
dicc_periodico_noticias['El Confidencial'] = noticias_periodico_ElConfidencial #ha llegado a la 33

# Recorro para cada periodico todas las noticias
for periodico in dicc_periodico_noticias:
    for noticia in dicc_periodico_noticias[periodico]:
        print(periodico + " - " + noticia)
        buscar_una_noticia(periodico, noticia)